In [1]:
import pandas as pd
import os
from tqdm import tqdm
from IPython.display import clear_output
from model.pipeline import FinancePipeline
from model.data_utils import download_raw_texts_from_urls, save_text_to, load_text_from    

In [3]:
directory_path = '/home/mkim/code/forecasting/stocks/raw_urls'
file_names = os.listdir(directory_path)

ticker = "aapl"
df = pd.read_csv(directory_path + f'/{ticker}_text.csv')[::-1]

data_dir = "/home/mkim/code/forecasting/stocks/summary"
pipe = FinancePipeline(data_dir)
pipe.set_prompts_for_ticker(ticker)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
2024-06-10 22:00:08.986899: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-10 22:00:08.988698: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-10 22:00:09.005180: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-10 22:00:09.426018: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: 

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [1]:
from transformers import AutoTokenizer, TextStreamer, AutoModelForCausalLM

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# AutoTokenizer.from_pretrained(model_name, token=token, device_map="auto")
model = AutoModelForCausalLM.from_pretrained(model_name, token=tokenizer, device_map="auto")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:

chat = [
    {"role": "user",
        "content": f"""For the given 4 historical values: 1.00 1.00 4.00 4.00, predict the future 4 values without producing any additional text"""},
    {"role": "assistant", "content": "The next 4 values are: 4.00 4.00 4.00 4.00"}]
content = chat + [{"role": "user",
        "content": f"""For the given 4 historical values: 3.00 2.00 4.00 5.00, predict the future 4 values without producing any additional text"""}]
prompt = [content]

new_prompt = tokenizer.apply_chat_template(
    prompt, tokenize=False)
model_inputs = tokenizer(
    new_prompt, return_tensors="pt", padding="longest")
# Get the device of the model
# model_inputs = model_inputs.to(device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

# Generate text using the model
generate_ids = model.generate(
    model_inputs.input_ids, max_new_tokens=150, eos_token_id=terminators, attention_mask=model_inputs.attention_mask)

output = tokenizer.batch_decode(
    generate_ids, skip_special_tokens=True)

ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

In [26]:
total_links = []

for path in sorted(os.listdir(directory_path)):
    print(path)
    df = pd.read_csv(directory_path + "/" + path)[::-1]
    total_links.append(len(df["timestamp"].unique()))

aapl_text.csv
abbv_text.csv
amd_text.csv
amzn_text.csv
avgo_text.csv
cost_text.csv
crm_text.csv
cvx_text.csv
goog_text.csv
hd_text.csv
jnj_text.csv
lly_text.csv
ma_text.csv
meta_text.csv
msft_text.csv
nflx_text.csv
nvda_text.csv
pg_text.csv
tsla_text.csv
unh_text.csv
v_text.csv
xom_text.csv


In [29]:
len(os.listdir(directory_path))

22

In [15]:
len(df["timestamp"].unique())

691

In [20]:
# date_str = "2024-04-29"
# date_str = "2024-04-28"
date_str = "2024-04-27"

current_urls = df[df["timestamp"] == date_str]["url"]

In [22]:
raw_texts = download_raw_texts_from_urls(current_urls)
save_text_to(raw_texts, f"summary/{ticker}_{date_str}_raw.txt")

# raw_texts = load_text_from(f"summary/{ticker}_{date_str}_raw.txt")
# raw_texts = [r for r in raw_texts if r != "<SEP>"]

100%|██████████| 14/14 [00:10<00:00,  1.27it/s]


In [23]:
raw_texts

["3 Stocks to Invest $30,000 in Right Now | The Motley Fool Please ensure Javascript is enabled for purposes of website accessibility Search ▲ S&P 500 +---% | ▲ Stock Advisor +---% Join The Motley Fool Accessibility Log In Help Accessibility Menu Our Services Investing Basics angle-down Investing 101 How to Invest Money What to Invest In How to Invest in Stocks How to Invest in Index Funds How to Invest in ETFs How to Invest in Real Estate Financial Dictionary Best Accounts Best Brokerage Accounts Best Brokers for Beginners Best IRA Accounts Best Roth IRA Accounts Best Options Brokers Best Crypto Apps Best Trading Apps Stock Market Basics Stock Market 101 Types of Stocks Stock Market Sectors Stock Market Indexes S&P 500 Dow Jones Nasdaq Composite Stock Market angle-down Stocks Growth Stocks Value Stocks Dividend Stocks Small Cap Stocks Large Cap Stocks Blue Chip Stocks How to Buy Stocks Industries to Invest In Consumer Goods Technology Energy Healthcare Financial Cryptocurrency NFTs Ne

In [24]:
summaries = []

for i, text in enumerate(tqdm(raw_texts, total=len(raw_texts))):
    clear_output(wait=True)
    print(f"Running {i} / {len(raw_texts)}")
    out = pipe.run_llama([pipe.summary_prompt, text + pipe.summary_ending_prompt])
    summaries.append(out)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Running 13 / 14
Here is a summary of the stock news specifically for Apple (AAPL):

**News:** Apple is reportedly in talks with Microsoft-backed OpenAI to integrate AI technology into the upcoming iPhone features, specifically iOS 18.

**Key Points:**

* Apple is discussing terms with OpenAI for potential integration of AI features into iOS 18.
* The companies had previously collaborated earlier this year, but there has been minimal engagement since then.
* Apple has not yet made a final decision on its partners and there is no guarantee that a deal will be reached.
* The iOS 18 update is set to include several new features based on Apple's in-house large language model AI model.
* Apple is also seeking partners to power a chatbot-like feature similar to OpenAI's ChatGPT.

**Market Impact:** Apple's shares closed 0.35% lower at $169.30 on Friday.

**Growth Trends:** Apple's AI push is expected to accelerate with the potential deal with OpenAI and Google, which could significantly impac

100%|██████████| 14/14 [02:19<00:00,  9.97s/it]


In [26]:
save_text_to(summaries, f"summary/{ticker}_{date_str}_summaries.txt")

# summaries = load_text_from(f"summary/{ticker}_{date_str}_summaries.txt")
# summaries = [s for s in summaries if s !="" and s != "<SEP>"]

In [27]:
summaries

["Here is a concise summary of the article:\n\nThe article discusses three stocks to invest $30,000 in right now, with $10,000 allocated to each stock. The stocks are:\n\n1. Advanced Micro Devices (AMD) - AMD has come a long way in the last decade, with its CPU market share rising from 18% to 33%. The company is investing heavily in AI and has an exciting outlook that could deliver major gains over the next decade.\n2. Amazon (AMZN) - Amazon has a lucrative hold on the e-commerce market and is investing heavily in AI. The company has a strong brand loyalty and has seen its annual revenue climb 546% since 2014.\n3. Apple (AAPL) - Apple is one of the most successful companies in tech history, with a vast and loyal user base. The company has stumbled over the last year, but its free cash flow hit $107 billion, making it well-equipped to keep investing in its business and come back strong.\n\nKey numbers and growth trends:\n\n* AMD's CPU market share has risen from 18% to 33% over the last

In [28]:
combined_summary = pipe.combine_summaries(summaries)

if len(combined_summary) == 1:
    final_summary = pipe.run_llama([pipe.filter_prompt, combined_summary[0]])

if len(combined_summary) > 1:
    print("Combinig summarys one more time...")
    combined_summary_again = pipe.combine_summaries(combined_summary)
    final_summary = pipe.run_llama([pipe.filter_prompt, combined_summary_again[0]])

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Running 0 / 3244, text_length = 20348
Here is a summarized version of the stock news specifically for Apple (AAPL):

**Summary:** Apple is a strong investment opportunity with its unique blend of qualities making it well-positioned to weather a stock market sell-off. The company's reasonable valuation, high-quality earnings, massive capital return program, and strong financial health make it an attractive investment opportunity.

**Recent News:** Apple is reportedly in talks with Microsoft-backed OpenAI to integrate AI technology into the upcoming iPhone features, specifically iOS 18. The companies had previously collaborated earlier this year, but there has been minimal engagement since then. Apple has not yet made a final decision on its partners and there is no guarantee that a deal will be reached.

**Market Impact:** Apple's shares closed 0.35% lower at $169.30 on Friday.

**Growth Trends:** Apple's AI push is expected to accelerate with the potential deal with OpenAI and Google, 

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Apple is a strong investment opportunity with its unique blend of qualities making it well-positioned to weather a stock market sell-off. The company's reasonable valuation, high-quality earnings, massive capital return program, and strong financial health make it an attractive investment opportunity.

Apple is reportedly in talks with Microsoft-backed OpenAI to integrate AI technology into the upcoming iPhone features, specifically iOS 18. The companies had previously collaborated earlier this year, but there has been minimal engagement since then. Apple has not yet made a final decision on its partners and there is no guarantee that a deal will be reached.

Apple's shares closed 0.35% lower at $169.30 on Friday.

Apple's AI push is expected to accelerate with the potential deal with OpenAI and Google, which could significantly impact the iPhone's AI capabilities. Apple's competitive advantages, including its ecosystem and brand loyalty, have driven its growth.

Bridgewater Associates

In [29]:
final_summary

"Apple is a strong investment opportunity with its unique blend of qualities making it well-positioned to weather a stock market sell-off. The company's reasonable valuation, high-quality earnings, massive capital return program, and strong financial health make it an attractive investment opportunity.\n\nApple is reportedly in talks with Microsoft-backed OpenAI to integrate AI technology into the upcoming iPhone features, specifically iOS 18. The companies had previously collaborated earlier this year, but there has been minimal engagement since then. Apple has not yet made a final decision on its partners and there is no guarantee that a deal will be reached.\n\nApple's shares closed 0.35% lower at $169.30 on Friday.\n\nApple's AI push is expected to accelerate with the potential deal with OpenAI and Google, which could significantly impact the iPhone's AI capabilities. Apple's competitive advantages, including its ecosystem and brand loyalty, have driven its growth.\n\nBridgewater A

In [30]:
save_text_to(final_summary, f"summary/{ticker}_{date_str}_final_summary.txt")

In [31]:
final_sentiment = pipe.run_llama(["Provide a sentiment for the following stock news:", final_summary])

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


The sentiment of this stock news is POSITIVE. The article highlights Apple's strong investment opportunities, including its reasonable valuation, high-quality earnings, and massive capital return program. The potential deal with OpenAI to integrate AI technology into the iPhone is also seen as a positive development, which could accelerate Apple's AI push and impact the iPhone's AI capabilities. The article also mentions Apple's strong performance and competitive advantages, which have driven its growth over the past 8 years, and notes that its valuation and growth trends indicate potential for continued growth. Overall, the tone of the article is optimistic and encouraging, suggesting that Apple is a strong investment opportunity.<|eot_id|>


"The sentiment of this stock news is POSITIVE. The article highlights Apple's strong investment opportunities, including its reasonable valuation, high-quality earnings, and massive capital return program. The potential deal with OpenAI to integrate AI technology into the iPhone is also seen as a positive development, which could accelerate Apple's AI push and impact the iPhone's AI capabilities. The article also mentions Apple's strong performance and competitive advantages, which have driven its growth over the past 8 years, and notes that its valuation and growth trends indicate potential for continued growth. Overall, the tone of the article is optimistic and encouraging, suggesting that Apple is a strong investment opportunity."